In [1]:
import torch
from torch import nn
from torchinfo import summary

from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from tqdm.auto import tqdm
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy  as np
import tracemalloc 

import json
import os
import glob
import pickle
from itertools import combinations
import gc
import time
import random

import warnings
warnings.filterwarnings("ignore")

/home/tran/miniconda3/envs/compgan/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
TEST = False

In [3]:
DATA_ROOT = "../data/compgan_dataset/"

train_data_file_name_ = "train_data{}.npy"
train_label_file_name_ = "train_label{}.npy"
test_data_file_name_ = "test_data{}.npy"
test_label_file_name_ = "test_label{}.npy"

TRAIN_FOLDER_PATH = os.path.join(DATA_ROOT, "train")
TEST_FOLDER_PATH = os.path.join(DATA_ROOT, "test")
RESULT_FOLDER_PATH = os.path.join(DATA_ROOT, "results")
PREV_CHECKPOINT_FOLDER_PATH = os.path.join(DATA_ROOT, "prev_checkpoint")

assert os.path.isdir(TRAIN_FOLDER_PATH) and os.path.isdir(TEST_FOLDER_PATH)
os.makedirs(RESULT_FOLDER_PATH, exist_ok=True)

data_files = sorted(glob.glob(os.path.join(TRAIN_FOLDER_PATH, train_data_file_name_.format("*"))))
print(data_files)
print(len(data_files))

['../data/compgan_dataset/train/train_data0.npy', '../data/compgan_dataset/train/train_data1.npy', '../data/compgan_dataset/train/train_data10.npy', '../data/compgan_dataset/train/train_data11.npy', '../data/compgan_dataset/train/train_data12.npy', '../data/compgan_dataset/train/train_data13.npy', '../data/compgan_dataset/train/train_data14.npy', '../data/compgan_dataset/train/train_data15.npy', '../data/compgan_dataset/train/train_data2.npy', '../data/compgan_dataset/train/train_data3.npy', '../data/compgan_dataset/train/train_data4.npy', '../data/compgan_dataset/train/train_data5.npy', '../data/compgan_dataset/train/train_data6.npy', '../data/compgan_dataset/train/train_data7.npy', '../data/compgan_dataset/train/train_data8.npy', '../data/compgan_dataset/train/train_data9.npy']
16


In [4]:
USER_NUM = 16
SENSOR_NUM = 7
EACH_SENSOR_CHANNEL = 6
assert USER_NUM == len(data_files)
feature_num = SENSOR_NUM * EACH_SENSOR_CHANNEL

In [5]:
# important

label_list = ['歩行(平地)',
 '歩行(階段)',
 'ベッド上での起き上がり',
 'ベッド椅子間の乗り移り(立つ)',
 'ベッド椅子間の乗り移り(立たない)',
 '立ち座り',
 '座位保持・座位バランス',
 '立位保持・立位バランス',
 '関節可動域増大訓練(肩)',
 '関節可動域増大訓練(股関節)']

label_dict = dict(enumerate(label_list))

In [6]:
# important
eng_label_dict = dict(zip(
    label_list,
    ['Walking', 'Upstair', 'Bed_Standup', 'Change_Bed', 'Change_Bed_Standup', 'Sit_Down', 'Sit', 'Stand', 'Shoulder_Exercise', 'Hip_Exercise']
))

eng_label_list = [eng_label_dict[i] for i in label_list]

In [7]:
# Update after 2023/11/11 houkoku
class CustomTrainDataset(Dataset):
    TRAIN_MODE = "train"
    TEST_MODE = "test"
    
    def __init__(self, mode, feature_data, label_data, missing_sensor_numbers=0):
        self.mode = mode
        assert mode in [self.TRAIN_MODE, self.TEST_MODE]
        
        self.features = feature_data
        self.label = label_data
        assert len(self.features) == len(self.label), "features len is not equal to label len"
        self.missing_sensor_numbers = missing_sensor_numbers

        self.missing_index_list = []
        for missing_count in range(missing_sensor_numbers + 1):
            for missing_index in combinations(range(SENSOR_NUM), missing_count):
                self.missing_index_list.append(missing_index)

    def transform(self, one_feature, missing_sensor_id_list):
        # Make one sensor data become 0
        one_feature_cp = one_feature.copy()
        
        for missing_sensor_id in missing_sensor_id_list:
            one_feature_cp[:, missing_sensor_id*6:(missing_sensor_id+1)*6] = 0
        return one_feature_cp
        
    def __len__(self):

        # take all available missing pattern * data number
        return len(self.features) * len(self.missing_index_list)
    
    def __getitem__(self, idx):
        # take all available missing pattern
        missing_sensor_id_list = self.missing_index_list[ idx // len(self.features) ]
        x = self.transform(self.features[ idx % len(self.features) ], missing_sensor_id_list)
        label = self.label[idx % len(self.features)]
        return x, int(label)


In [8]:
class DataModule(pl.LightningDataModule):
    STANDARDIZE = False
    
    def __init__(self, test_user, missing_sensor_numbers, batch_size=2048):
        super().__init__()
        self.test_user = test_user
        self.missing_sensor_numbers = missing_sensor_numbers
        self.batch_size = batch_size
        self.scaler = None

    def load_data(self, mode):
        if mode == "train":
            folder_path = TRAIN_FOLDER_PATH
            data_file_name = train_data_file_name_
            label_file_name = train_label_file_name_

            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            self.scaler = StandardScaler()
            # train_val_data = self.scaler.fit_transform(train_val_data.reshape(l, -1)).reshape(l, s, d, w)
            train_val_data = train_val_data.reshape(l, s ,d, w).transpose(0, 3, 1, 2)
            
            # train_val_data_list = None
            # train_val_label_list = None
            # for user in range(USER_NUM):
            #     if user == self.test_user: 
            #         continue
                
            #     print("train_user", user)
            #     train_data_file_name = data_file_name.format(user)
            #     train_label_file_name = label_file_name.format(user)

            #     train_data_file_path = os.path.join(folder_path, train_data_file_name)
            #     train_label_file_path = os.path.join(folder_path, train_label_file_name)
                
            #     train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            #     l, s, d, w = train_val_data.shape
            #     train_val_data = train_val_data.reshape(l, s ,d, w).transpose(0, 3, 1, 2)

            #     if train_val_data_list is None:
            #         train_val_data_list = train_val_data
            #         train_val_label_list = train_val_label
            #     else:
            #         train_val_data_list = np.vstack((train_val_data_list, train_val_data))
            #         train_val_label_list = np.concatenate((train_val_label_list, train_val_label))

            # train_val_data = train_val_data_list
            # train_val_label = train_val_label_list
            
        elif mode == "test":
            folder_path = TEST_FOLDER_PATH
            data_file_name = test_data_file_name_
            label_file_name = test_label_file_name_
    
            train_data_file_name = data_file_name.format(self.test_user)
            train_label_file_name = label_file_name.format(self.test_user)
    
            train_data_file_path = os.path.join(folder_path, train_data_file_name)
            train_label_file_path = os.path.join(folder_path, train_label_file_name)
            train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
            l, s, d, w = train_val_data.shape
            # train_val_data = self.scaler.transform(train_val_data.reshape(l, -1)).reshape(l, s, d, w)
            train_val_data = train_val_data.reshape(l, s ,d, w).transpose(0, 3, 1, 2)
    
        return train_val_data, train_val_label

    def setup(self, stage: str):
        # Assign Train/val split(s) for use in Dataloaders
        
        if stage == "validate" or stage == "fit":
            train_val_data, train_val_label = self.load_data("train")
            self.train_data, self.val_data, self.train_label, self.val_label = train_test_split(
                train_val_data, train_val_label, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

            self.train_dataset = CustomTrainDataset(
                CustomTrainDataset.TRAIN_MODE, self.train_data, self.train_label, missing_sensor_numbers=self.missing_sensor_numbers)
            self.val_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.val_data, self.val_label, missing_sensor_numbers=self.missing_sensor_numbers)

        elif stage == "test" or stage == "predict":
            train_val_data, train_val_label = self.load_data("test")
            self.test_data = train_val_data
            self.test_label = train_val_label

            self.test_dataset = CustomTrainDataset(
                CustomTrainDataset.TEST_MODE, self.test_data, self.test_label, missing_sensor_numbers=self.missing_sensor_numbers)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=4, shuffle=True, pin_memory=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

    def teardown(self, stage):
        print("teardown")
        if stage == "validate" or stage == "fit":
            del self.train_data, self.train_label
            del self.val_data, self.val_label
            del self.train_dataset
            del self.val_dataset
            
        elif stage == "test" or stage == "predict":
            del self.test_data, self.test_label
            del self.test_dataset
        gc.collect()


In [9]:
if TEST:
    data_module = DataModule(test_user=0, missing_sensor_numbers=1)
    data_module.setup("fit")
    print(data_module.train_data.shape)
    print(data_module.val_data.shape)
    # print(data_module.test_data.shape)

In [10]:
if TEST:
    data = data_module.val_dataset[1273 * 2 + 1]
    print(data[1], data[0][0])
    print(len(data_module.val_dataset))

In [11]:
if TEST:
    del data_module, data
    gc.collect()

## MODEL DEFINITION

In [12]:

class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
            
        Input: ()
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))



In [13]:

class ConvLSTM(nn.Module):

    """

    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.

    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [14]:
class ConvLSTMModel(pl.LightningModule):
    def __init__(self, hidden_size=64, sequence_length=256, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(10, sequence_length, SENSOR_NUM, EACH_SENSOR_CHANNEL)
        
        self.conv_lstm = ConvLSTM(input_dim=1, hidden_dim=[cnn_filter_size], kernel_size=(1,3), num_layers=1, batch_first=True,)
        self.dropout = nn.Dropout2d(p=0.5)
        self.linear1 = nn.Linear(in_features=sequence_length * cnn_filter_size * SENSOR_NUM* EACH_SENSOR_CHANNEL, out_features=hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(in_features=hidden_size, out_features=output_size)
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        out = torch.unsqueeze(x, dim=2)
        h, _ = self.conv_lstm(out)
        out = self.dropout(h[0])
        out = out.view(out.shape[0], self.hparams.sequence_length * self.hparams.cnn_filter_size * SENSOR_NUM * EACH_SENSOR_CHANNEL)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.softmax(out)
        return out        
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        
        

In [15]:
if TEST:
    from pytorch_lightning.utilities.model_summary import ModelSummary
    model = ConvLSTMModel()
    model_summary = ModelSummary(model, max_depth=2)
    print(model_summary)

In [16]:
# x = torch.rand(10, 256, 7, 6)
# conv_lstm = ConvLSTM(input_dim=1, hidden_dim=[64], kernel_size=(1,3), num_layers=1, batch_first=True)
# x = torch.unsqueeze(x, dim=2)
# print(conv_lstm(x)[0][0].shape)
# out = conv_lstm(x)[0][0].view(-1, 256, 64 * 42)

# attention1 = nn.MultiheadAttention(
#             embed_dim=42*64,
#             num_heads=1,
#             batch_first=True
#         )
# print(attention1(out, out, out)[0].shape)

In [17]:
# hidden_size=64
# sequence_length=256
# cnn_filter_size=64
# output_size=10

# x = torch.Tensor(10, sequence_length, SENSOR_NUM, EACH_SENSOR_CHANNEL)
# conv_lstm = ConvLSTM(input_dim=1, hidden_dim=[cnn_filter_size], kernel_size=(1,3), num_layers=1, batch_first=True,)
# dropout = nn.Dropout2d(p=0.5)
# attention1 = nn.MultiheadAttention(
#             embed_dim=cnn_filter_size * SENSOR_NUM * EACH_SENSOR_CHANNEL,
#             num_heads=1,
#             batch_first=True
#         )
# linear1 = nn.Linear(in_features=sequence_length * cnn_filter_size * SENSOR_NUM* EACH_SENSOR_CHANNEL, out_features=hidden_size)
# relu = nn.ReLU()
# linear2 = nn.Linear(in_features=hidden_size, out_features=output_size)
# softmax = nn.Softmax()

# out = torch.unsqueeze(x, dim=2)
# h, _ = conv_lstm(out)
# out = dropout(h[0])
# out = out.view(out.shape[0], sequence_length, cnn_filter_size * SENSOR_NUM* EACH_SENSOR_CHANNEL)
# out, _ = attention1(out, out, out)
# out = out.reshape(-1, sequence_length * cnn_filter_size * SENSOR_NUM* EACH_SENSOR_CHANNEL)
# out = linear1(out)
# out = relu(out)
# out = linear2(out)
# out = softmax(out)

In [18]:
class ConvLSTMAttentionModel(pl.LightningModule):
    def __init__(self, hidden_size=64, sequence_length=256, cnn_filter_size=64, output_size=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.example_input_array = torch.Tensor(10, sequence_length, SENSOR_NUM, EACH_SENSOR_CHANNEL)
        
        self.conv_lstm = ConvLSTM(input_dim=1, hidden_dim=[cnn_filter_size], kernel_size=(1,3), num_layers=1, batch_first=True,)
        self.dropout = nn.Dropout2d(p=0.5)
        self.attention1 = nn.MultiheadAttention(
            embed_dim=cnn_filter_size * SENSOR_NUM * EACH_SENSOR_CHANNEL,
            num_heads=1,
            batch_first=True
        )
        self.linear1 = nn.Linear(in_features=sequence_length * cnn_filter_size * SENSOR_NUM* EACH_SENSOR_CHANNEL, out_features=hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(in_features=hidden_size, out_features=output_size)
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        out = torch.unsqueeze(x, dim=2)
        h, _ = self.conv_lstm(out)
        out = self.dropout(h[0])
        out = out.view(out.shape[0], self.hparams.sequence_length, self.hparams.cnn_filter_size * SENSOR_NUM* EACH_SENSOR_CHANNEL)
        out, _ = self.attention1(out, out, out)
        out = out.reshape(-1, self.hparams.sequence_length * self.hparams.cnn_filter_size * SENSOR_NUM* EACH_SENSOR_CHANNEL)
        out = self.linear1(out)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.softmax(out)
        return out        
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(params=self.parameters(), lr=0.0005)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        # 1. Forward pass
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("train_loss", loss, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        # this is the test loop
        X, y = batch
        X = X.float()
        y = y
    
        # 1. Forward pass
        test_pred_logits = self.forward(X)

        # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc = ((test_pred_labels == y).sum().item()/len(test_pred_labels))
        self.log("test_acc", test_acc)
        
    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        X, y = batch
        X = X.float()
        y = y
        
        y_pred = self.forward(X)
        # 2. Calculate  and accumulate loss
        loss = F.cross_entropy(y_pred, y)
        
        self.log("val_loss", loss, prog_bar=True)

    def predict_step(self, batch, batch_idx):
        X, y = batch
        X = X.float()
        
        test_pred_logits = self.forward(X)
        test_pred_labels = test_pred_logits.argmax(dim=1)
        
        return test_pred_labels
        

In [20]:
from pytorch_lightning.utilities.model_summary import ModelSummary
model = ConvLSTMAttentionModel()
model_summary = ModelSummary(model, max_depth=2)
print(model_summary)

  | Name                | Type                            | Params | In sizes                                            | Out sizes                                                  
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0 | conv_lstm           | ConvLSTM                        | 50.2 K | [10, 256, 1, 7, 6]                                  | [[[10, 256, 64, 7, 6]], [[[10, 64, 7, 6], [10, 64, 7, 6]]]]
1 | conv_lstm.cell_list | ModuleList                      | 50.2 K | ?                                                   | ?                                                          
2 | dropout             | Dropout2d                       | 0      | [10, 256, 64, 7, 6]                                 | [10, 256, 64, 7, 6]                                        
3 | attention1          | MultiheadAttention              | 28.9 M | [[10, 256,

In [21]:
# user 0: tran
# user 1: hachix
# train_model_class = [CNNLSTMAttentionModel, CNNAttentionModel, LSTMAttentionModel, CNNModel, LSTMModel, CNNLSTMModel]
# train_model_class = [ConvLSTMModel, ConvLSTMAttentionModel]
train_model_class = [ConvLSTMModel]



In [22]:
# only for test

def test():
    patience = 20
    missing_sensor_numbers = 4 # no missing sensor
    user = 2 # use user2 to test
    batch_size_dict = {ConvLSTMAttentionModel:128, ConvLSTMModel:256}
    for model_class in train_model_class:
        data_module = DataModule(test_user=user, 
                                 missing_sensor_numbers=missing_sensor_numbers, 
                                 batch_size=batch_size_dict.get(model_class, 1024))
        
        model = model_class(input_size=42, output_size=10)
        
        model_name = model.__class__.__name__
        print("Running for model", model_name)
        print("summary(model)", summary(model))

        tb_logger = TensorBoardLogger(f"./loggers/{model_name}")
        
        trainer = pl.Trainer(
            logger=tb_logger,
            callbacks=[EarlyStopping(monitor="val_loss", patience=patience, mode="min")],
            fast_dev_run = True,
            precision="16-mixed"
        )
        trainer.fit(model, data_module)
        trainer.test(model, data_module)
if TEST:
    test()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Running for model ConvLSTMAttentionModel
summary(model) ===========================================================================
Layer (type:depth-idx)                             Param #
ConvLSTMAttentionModel                             --
├─ConvLSTM: 1-1                                    --
│    └─ModuleList: 2-1                             --
│    │    └─ConvLSTMCell: 3-1                      50,176
├─Dropout2d: 1-2                                   --
├─MultiheadAttention: 1-3                          21,684,096
│    └─NonDynamicallyQuantizableLinear: 2-2        7,228,032
├─Linear: 1-4                                      44,040,256
├─ReLU: 1-5                                        --
├─Linear: 1-6                                      650
├─Softmax: 1-7                                     --
Total params: 73,003,210
Trainable params: 73,003,210
Non-trainable params: 0


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params | In sizes                                            | Out sizes                                                  
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
0 | conv_lstm  | ConvLSTM           | 50.2 K | [10, 256, 1, 7, 6]                                  | [[[10, 256, 64, 7, 6]], [[[10, 64, 7, 6], [10, 64, 7, 6]]]]
1 | dropout    | Dropout2d          | 0      | [10, 256, 64, 7, 6]                                 | [10, 256, 64, 7, 6]                                        
2 | attention1 | MultiheadAttention | 28.9 M | [[10, 256, 2688], [10, 256, 2688], [10, 256, 2688]] | [[10, 256, 2688], [10, 256, 256]]                          
3 | linear1    | Linear             | 44.0 M | [10, 688128]                                        | [10, 64]                                     

Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it, train_loss=2.300, val_loss=2.300]

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  1.20s/it, train_loss=2.300, val_loss=2.300]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


teardown
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  4.02it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    0.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
teardown


In [33]:
# patience = 100
# missing_sensor_numbers = 0
# all_test_pred = {}
batch_size_dict = {ConvLSTMAttentionModel:128, ConvLSTMModel:256}
for model_class in train_model_class: 
    
    model = model_class(output_size=10)
    model_name = model.__class__.__name__
    print("Running for model", model_name)
    
    for missing_sensor_numbers in [0]: ## Changed for 1 missing sensor
        for user in [0]: ## Changed for user 2 only

            # Load from exist previous training cpkt for continuous learning & testing
            # previous_checkpoint_file = os.path.join(PREV_CHECKPOINT_FOLDER_PATH, f"{model_name}_{missing_sensor_numbers}missing_user{user}.ckpt")
            previous_checkpoint_file = "/home/tran/acttivity_recognition/data/compgan_dataset/results/logger90_0missing/ConvLSTMModel/user0/version_3/checkpoints/last-epoch=199-val_loss=1.54-val_acc=0.00.ckpt"
            
            if os.path.isfile(previous_checkpoint_file):
                model = model_class.load_from_checkpoint(previous_checkpoint_file)
                print("loaded", model_name, missing_sensor_numbers, user)
                
            start_timer = time.perf_counter()
            print(f"\n*************training on User{user}*************")
            
            data_module = DataModule(test_user=user, 
                                     missing_sensor_numbers=missing_sensor_numbers,
                                     batch_size=batch_size_dict.get(model_class, 2048))

            save_dir = os.path.join(RESULT_FOLDER_PATH, f"./logger90_{missing_sensor_numbers}missing/{model_name}")
            save_dir_name = f"user{user}"
            
            tb_logger = TensorBoardLogger(save_dir=save_dir, name=save_dir_name)
           
            model_checkpoint = ModelCheckpoint(
                save_top_k=1,  # get the 1 minimum val loss checkpoint
                monitor="val_loss",
                mode="min",
                filename="val_loss_min-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}"
            )
            
            model_checkpoint_save_last = ModelCheckpoint(
                save_top_k=1,  # get the 1 minimum val loss checkpoint
                filename="last-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}"
            )
            
            trainer = pl.Trainer(
                logger=tb_logger,
                callbacks=[
                    # EarlyStopping(monitor="val_loss", patience=patience, mode="min"),
                    LearningRateMonitor("epoch"),
                    model_checkpoint,
                    model_checkpoint_save_last
                ],
                precision="16-mixed",
                accumulate_grad_batches=1,
                log_every_n_steps=10,
                check_val_every_n_epoch=5,
                max_epochs=500,
            )
        
            trainer.fit(model, data_module)
            trainer.test(model, data_module)
    
            end_timer = time.perf_counter()
            exec_time = end_timer - start_timer

            trainer_test_dict = trainer.logged_metrics
            for key in trainer_test_dict.keys():
                trainer_test_dict[key] = trainer_test_dict[key].item()
            trainer_test_dict["exec_time"] = int(exec_time)
            
            with open(os.path.join(trainer.log_dir, f"result.json"), "w") as f:
                json.dump(trainer_test_dict, f)


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params | In sizes                                         | Out sizes                                                      
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
0 | conv_lstm  | ConvLSTM           | 246 K  | [1024, 1, 256, 7, 6]                             | [[[1024, 1, 64, 7, 6]], [[[1024, 64, 7, 6], [1024, 64, 7, 6]]]]
1 | dropout    | Dropout2d          | 0      | [1024, 1, 64, 7, 6]                              | [1024, 1, 64, 7, 6]                                            
2 | attention1 | MultiheadAttention | 7.2 K  | [[1024, 64, 42], [1024, 64, 42], [1024, 64, 42]] | [[1024, 6

Running for model ConvLSTMAttentionModel
loaded ConvLSTMAttentionModel 6 0

*************training on User0*************
Epoch 2:  63%|██████▎   | 398/632 [00:08<00:04, 48.89it/s, v_num=10, train_loss=2.150]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 27/27 [00:00<00:00, 67.34it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.30443739891052246
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
teardown


## Predict full length

In [22]:
# # Update after 2023/11/11 houkoku
# class CustomTestDataset(Dataset):
#     def __init__(self, feature_data, label_data, missing_sensor_numbers=0):
        
#         self.features = feature_data
#         self.label = label_data
#         assert len(self.features) == len(self.label), "features len is not equal to label len"
#         self.missing_sensor_numbers = missing_sensor_numbers

#         self.missing_index_list = []
        
#         for missing_index in combinations(range(SENSOR_NUM), missing_sensor_numbers):
#             self.missing_index_list.append(missing_index)

#     def transform(self, one_feature, missing_sensor_id_list):
#         # Make one sensor data become 0
#         one_feature_cp = one_feature.copy()
        
#         for missing_sensor_id in missing_sensor_id_list:
#             one_feature_cp[:, missing_sensor_id*6:(missing_sensor_id+1)*6] = 0
#         return one_feature_cp
        
#     def __len__(self):

#         # take all available missing pattern * data number
#         return len(self.features) * len(self.missing_index_list)
    
#     def __getitem__(self, idx):
#         # take all available missing pattern
#         missing_sensor_id_list = self.missing_index_list[ idx // len(self.features) ]
#         x = self.transform(self.features[ idx % len(self.features) ], missing_sensor_id_list)
#         label = self.label[idx % len(self.features)]
#         return x, int(label)


In [23]:
# class DataModule(pl.LightningDataModule):
#     STANDARDIZE = False
    
#     def __init__(self, test_user, missing_sensor_numbers, batch_size=2048):
#         super().__init__()
#         self.test_user = test_user
#         self.missing_sensor_numbers = missing_sensor_numbers
#         self.batch_size = batch_size
#         self.scaler = None

#     def load_data(self, mode):
#         if mode == "train":
#             folder_path = TRAIN_FOLDER_PATH
#             data_file_name = train_data_file_name_
#             label_file_name = train_label_file_name_

#             train_data_file_name = data_file_name.format(self.test_user)
#             train_label_file_name = label_file_name.format(self.test_user)
    
#             train_data_file_path = os.path.join(folder_path, train_data_file_name)
#             train_label_file_path = os.path.join(folder_path, train_label_file_name)
#             train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
#             l, s, d, w = train_val_data.shape
#             self.scaler = StandardScaler()
#             # train_val_data = self.scaler.fit_transform(train_val_data.reshape(l, -1)).reshape(l, s, d, w)
#             train_val_data = train_val_data.reshape(l, s ,d, w).transpose(0, 3, 1, 2)
            
#         elif mode == "test":
#             folder_path = TEST_FOLDER_PATH
#             data_file_name = test_data_file_name_
#             label_file_name = test_label_file_name_
    
#             train_data_file_name = data_file_name.format(self.test_user)
#             train_label_file_name = label_file_name.format(self.test_user)
    
#             train_data_file_path = os.path.join(folder_path, train_data_file_name)
#             train_label_file_path = os.path.join(folder_path, train_label_file_name)
#             train_val_data, train_val_label = np.load(train_data_file_path), np.load(train_label_file_path)
#             l, s, d, w = train_val_data.shape
#             # train_val_data = self.scaler.transform(train_val_data.reshape(l, -1)).reshape(l, s, d, w)
#             train_val_data = train_val_data.reshape(l, s ,d, w).transpose(0, 3, 1, 2)
    
#         return train_val_data, train_val_label

#     def setup(self, stage: str):
#         # Assign Train/val split(s) for use in Dataloaders
        
#         if stage == "validate" or stage == "fit":
#             train_val_data, train_val_label = self.load_data("train")
#             self.train_data, self.val_data, self.train_label, self.val_label = train_test_split(
#                 train_val_data, train_val_label, test_size=0.2, train_size=0.8, random_state=42, shuffle=True)

#             self.train_dataset = CustomTrainDataset(
#                 CustomTrainDataset.TRAIN_MODE, self.train_data, self.train_label, missing_sensor_numbers=self.missing_sensor_numbers)
#             self.val_dataset = CustomTrainDataset(
#                 CustomTrainDataset.TEST_MODE, self.val_data, self.val_label, missing_sensor_numbers=self.missing_sensor_numbers)

#         elif stage == "test" or stage == "predict":
#             train_val_data, train_val_label = self.load_data("test")
#             self.test_data = train_val_data
#             self.test_label = train_val_label

#             self.test_dataset = CustomTrainDataset(
#                 CustomTrainDataset.TEST_MODE, self.test_data, self.test_label, missing_sensor_numbers=self.missing_sensor_numbers)
    
#     def train_dataloader(self):
#         return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=4, shuffle=True, pin_memory=True)
    
#     def val_dataloader(self):
#         return DataLoader(self.val_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)
    
#     def test_dataloader(self):
#         return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

#     def predict_dataloader(self):
#         return DataLoader(self.test_dataset, batch_size=self.batch_size,  num_workers=4, shuffle=False, pin_memory=True)

#     def teardown(self, stage):
#         print("teardown")
#         if stage == "validate" or stage == "fit":
#             del self.train_data, self.train_label
#             del self.val_data, self.val_label
#             del self.train_dataset
#             del self.val_dataset
            
#         elif stage == "test" or stage == "predict":
#             del self.test_data, self.test_label
#             del self.test_dataset
#         gc.collect()


In [24]:
# all_test_pred = {}

# for missing_sensor_numbers in range(7): ## Changed for 1 missing sensor
#     all_test_pred[missing_sensor_numbers] = {}
    
#     for model_class in train_model_class: 
#         all_user_y_true = []
#         all_user_y_pred = []
        
#         for user in range(1): ## Changed for user 2 only
#             model_name = model_class.__name__
#             checkpoint_path = os.path.join(RESULT_FOLDER_PATH, f"./logger90_6missing/{model_name}", f"user{user}", "version_10", "checkpoints", "last*.ckpt")
#             checkpoint_glob = glob.glob(checkpoint_path)
#             assert len(checkpoint_glob) == 1, checkpoint_path
#             checkpoint_file = checkpoint_glob[0]

#             assert os.path.exists(checkpoint_file), checkpoint_file
#             model = model_class.load_from_checkpoint(checkpoint_file)

#             data_module = DataModule(test_user=user, missing_sensor_numbers=missing_sensor_numbers, batch_size=512)
#             data_module.setup("test")
#             test_loader = data_module.test_dataloader()
            
#             tb_logger = TensorBoardLogger(os.path.join(RESULT_FOLDER_PATH, f"./logger90_6missing/{model_name}", f"user{user}", "version_10", f"result_test"))
#             trainer = pl.Trainer(logger=tb_logger)

#             pred = trainer.predict(model, test_loader)

#             y_true_list = []
#             for batch_idx, test_data in enumerate(test_loader):
#                 _, y_true = test_data
#                 y_true_list.append(y_true)

#             all_user_y_true.append(torch.cat(y_true_list))
#             all_user_y_pred.append(torch.cat(pred))
        
#         print("model_name", model_name)
#         all_test_cpu = list(map(lambda x: x.cpu().item(), torch.cat(all_user_y_true)))
#         all_pred_cpu = list(map(lambda x: x.cpu().item(), torch.cat(all_user_y_pred)))
        
#         all_test_pred[missing_sensor_numbers][model_name] = (all_test_cpu, all_pred_cpu)
        
#         print("accuracy", accuracy_score(all_test_cpu, all_pred_cpu))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: ../data/compgan_dataset/results/./logger90_6missing/ConvLSTMAttentionModel/user0/version_7/result_test/lightning_logs
2023-12-14 17:24:41.243362: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-14 17:24:41.276700: I tensorflow/core/pla

Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model_name ConvLSTMAttentionModel
accuracy 0.5233644859813084
Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 95.50it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

model_name ConvLSTMAttentionModel
accuracy 0.4392523364485981
Predicting DataLoader 0: 100%|██████████| 42/42 [00:00<00:00, 106.76it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model_name ConvLSTMAttentionModel
accuracy 0.41612385537619184
Predicting DataLoader 0: 100%|██████████| 51/51 [00:00<00:00, 124.88it/s]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model_name ConvLSTMAttentionModel
accuracy 0.40062305295950157
Predicting DataLoader 0: 100%|██████████| 54/54 [00:00<00:00, 119.62it/s]
model_name ConvLSTMAttentionModel
accuracy 0.393847965266024


In [25]:
# df = pd.DataFrame(index=[f"missing_{x}" for x in range(SENSOR_NUM)])

# for model_class in train_model_class: 
#     acc_list = []
#     for missing_sensor_numbers in range(7): ## Changed for 1 missing sensor
#         for user in range(1):
#             all_test_cpu, all_pred_cpu = all_test_pred[missing_sensor_numbers][model_class.__name__]
#             acc = f1_score(all_test_cpu, all_pred_cpu, average="macro")
#             acc_list.append(acc)
            
#     df[model_class.__name__] = acc_list
# print(df)
        

           ConvLSTMAttentionModel
missing_0                0.278775
missing_1                0.257344
missing_2                0.236514
missing_3                0.217152
missing_4                0.201297
missing_5                0.190957
missing_6                0.186530
